In [24]:
ON_COLAB = True
if ON_COLAB:
    import os
    os.chdir('/content/')
    !pip install -q datasets emoji accelerate sklearn
    !git clone https://github.com/linv24/da-stance-detection.git
    os.chdir('/content/da-stance-detection')

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'da-stance-detection'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 249 (delta 50), reused 227 (delta 31), pack-reused 0
Receiving objects: 100% (249/249), 1.91 MiB | 15.87 MiB/s, done.
Resolving deltas: 100% (50/50), done.
Filtering content: 100% (116/116), 3.53 GiB | 143.07 MiB/s, done.


In [3]:
import torch
import pandas as pd
import transformers
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset
from sklearn.metrics import f1_score
import numpy as np

from data import data_util

In [160]:
ds_csv = 'augment/data/llama_augmented.csv'

NUM_LABELS = 3
CHECKPOINT_OUTPUT_PATH = 'checkpoint/'

In [161]:
ds = load_dataset('csv', data_files=ds_csv, split='train')
split_ds = ds.train_test_split(test_size=0.2)
split_ds

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Tweet', 'Target', 'Stance'],
        num_rows: 3628
    })
    test: Dataset({
        features: ['Tweet', 'Target', 'Stance'],
        num_rows: 907
    })
})

In [162]:
tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base')

In [163]:
def preprocess_data(batch_tweet):
    encoding = tokenizer(batch_tweet['Tweet'],
                         return_tensors='pt',
                         truncation=True,
                         max_length=128,
                         padding='max_length')
    encoding['label'] = batch_tweet['Stance']
    return encoding

In [164]:
encoded_split_ds = split_ds.map(preprocess_data,
                                batched=True)
encoded_split_ds

Map:   0%|          | 0/3628 [00:00<?, ? examples/s]

Map:   0%|          | 0/907 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Tweet', 'Target', 'Stance', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 3628
    })
    test: Dataset({
        features: ['Tweet', 'Target', 'Stance', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 907
    })
})

In [165]:
tokenizer.decode(encoded_split_ds['train'][0]['input_ids'])

'<s> Use our tax money to help our communities, stop defunding schools. </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [166]:
model = AutoModelForSequenceClassification.from_pretrained('vinai/bertweet-base',
                                                           num_labels=NUM_LABELS)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [167]:
def compute_metrics(eval_preds):
    preds = eval_preds.predictions
    preds = np.argmax(preds, axis=1)
    labels = eval_preds.label_ids

    scores = f1_score(preds, labels, average='macro')

    against_preds = []
    favor_preds = []
    for ix, label in enumerate(labels):
        if label == 0: # AGAINST
            against_preds.append(preds[ix])
        elif label == 1: # FAVOR
            favor_preds.append(preds[ix])

    f_against = f1_score([0] * len(against_preds), against_preds, average='macro')
    f_favor = f1_score([1] * len(favor_preds), favor_preds, average='macro')



    return {
        'f_against': f_against,
        'f_favor': f_favor,
        'f_avg': (f_against + f_favor) / 2
    }


In [168]:
# model hyperparameters

lr = 2e-5
batch_size = 32
num_epochs = 10
seed = 42

In [169]:
training_args = TrainingArguments(
    output_dir=CHECKPOINT_OUTPUT_PATH + ds_csv,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    seed=seed,
    load_best_model_at_end=True,
    # metric_for_best_model="f1",
    optim="adamw_torch",
)

In [170]:
trainer = Trainer(model=model, args=training_args,
                  train_dataset=encoded_split_ds['train'],
                  eval_dataset=encoded_split_ds['test'],
                  compute_metrics=compute_metrics
                  )

In [171]:
trainer.model_init

In [172]:
trainer.train()

Epoch,Training Loss,Validation Loss,F Against,F Favor,F Avg
1,0.576300,0.415799,0.466374,0.431866,0.449120
2,0.362000,0.317119,0.429688,0.482824,0.456256
3,0.251800,0.297505,0.483734,0.450861,0.467298
4,0.188300,0.295101,0.459259,0.478846,0.469053
5,0.166300,0.274509,0.463235,0.468627,0.465931
6,0.141000,0.302866,0.478571,0.461233,0.469902
7,0.121000,0.347599,0.477077,0.456914,0.466996
8,0.099100,0.353686,0.456845,0.476329,0.466587
9,0.086400,0.336490,0.467153,0.475315,0.471234
10,0.073100,0.352093,0.470247,0.470703,0.470475


TrainOutput(global_step=1140, training_loss=0.20652561438711067, metrics={'train_runtime': 304.8469, 'train_samples_per_second': 119.011, 'train_steps_per_second': 3.74, 'total_flos': 2386438698792960.0, 'train_loss': 0.20652561438711067, 'epoch': 10.0})

In [173]:
# test_ds = load_dataset('csv', data_files='data/semeval-2016/test.csv', split='train')
test_ds = load_dataset('csv', data_files='data/original_test.csv', split='train')
test_ds

Dataset({
    features: ['Tweet', 'Target', 'Stance'],
    num_rows: 2914
})

In [174]:
test_ds[0]

{'Tweet': '@tedcruz And, #HandOverTheServer she wiped clean + 30k deleted emails, explains dereliction of duty/lies re #Benghazi,etc #tcot',
 'Target': 'Hillary Clinton',
 'Stance': 0}

In [175]:
encoded_test_ds = test_ds.map(preprocess_data,
                                      batched=True)
encoded_test_ds

Map:   0%|          | 0/2914 [00:00<?, ? examples/s]

Dataset({
    features: ['Tweet', 'Target', 'Stance', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 2914
})

In [176]:
out = trainer.evaluate(encoded_test_ds)
out

{'eval_loss': 2.938250780105591,
 'eval_f_against': 0.40814594823928724,
 'eval_f_favor': 0.21725571725571724,
 'eval_f_avg': 0.31270083274750227,
 'eval_runtime': 6.6766,
 'eval_samples_per_second': 436.449,
 'eval_steps_per_second': 13.779,
 'epoch': 10.0}

In [177]:
def eval_pipeline(csv_files, test_csv, tokenizer):
    for ds_csv in csv_files:
        ds = load_dataset('csv', data_files=ds_csv, split='train')
        split_ds = ds.train_test_split(test_size=0.2)

        encoded_split_ds = split_ds.map(preprocess_data,
                                batched=True)

        model = AutoModelForSequenceClassification.from_pretrained('vinai/bertweet-base',
                                                           num_labels=NUM_LABELS)
        trainer = Trainer(model=model, args=training_args,
                  train_dataset=encoded_split_ds['train'],
                  eval_dataset=encoded_split_ds['test'],
                  compute_metrics=compute_metrics
                  )

        trainer.train()

        # evaluate
        test_ds = load_dataset('csv', data_files=test_csv, split='train')
        encoded_test_ds = test_ds.map(preprocess_data,
                                      batched=True)

        # out = trainer.model(**encoding)

